# Atendimento ao Cliente - Analise de sentimentos 

## Resumo

Na no canal de atendimento ao cliente da Vammo existe uma pergunta de campo aberto que possibilita o avaliador adicionar um comentário (campo aberto). 

O objetivo é utilizar a [API Google NLP](https://cloud.google.com/natural-language/docs/analyzing-sentiment?hl=pt-br) (modelo pré-treinado) para identificar o `sentimento` dos comentários, onde cada comentário terá uma polaridade atribuída conforme abaixo:

<p align="center">
<img src="https://centricconsulting.com/wp-content/uploads/2022/08/Blog_DA_Sentiment_Customer_08052022.png" alt="drawing" width="500" />
</p>

> Obs.: Não será realizada análise de emoções. Análise de emoções é um método utilizado para classificar uma emoções específicas, como raiva, surpresa, felicidade e etc, presentes em um comentário.



**Importar bibliotecas**

In [3]:
import os
import pandas as pd
from google.oauth2 import service_account
from google.cloud import language_v2, storage
import numpy as np

**Credenciamento na conta de serviço do google**

In [4]:
# Usar as credenciais para criar um cliente do Google Cloud Storage
client = storage.Client()

**Extraindo os dados e filtrando atendimentos comentados**

In [12]:
# Extraindo comentarios
print("Extraindo comentarios..")

# Dirertorio de tabelas auxiliares
aux_file_path = os.path.join(os.getcwd(), 'tabelas_auxiliares') 

# Caminho do arquivo
file_path = os.path.join(aux_file_path, 'atendimento_last2m_tratado_bool_outliers.xlsx')

# Carregar o arquivo CSV
df_orig = pd.read_excel(file_path)

# Filtrar as colunas desejadas
df = df_orig[['id', 'satisfaction_Answer', 'satisfaction_rating', 'satisfaction_status']]

# Selecionar apenas as linhas onde satisfaction_status é 'commented'
df = df[df['satisfaction_status'] == 'commented']

# Remover a coluna 'satisfaction_status' já que não é mais necessária
df = df.drop(columns=['satisfaction_status'])

# Renomear as colunas
df = df.rename(columns={
    'id': 'token',
    'satisfaction_rating': 'csat',
    'satisfaction_Answer': 'text'
})

# Exibir o DataFrame filtrado
df.head(4)

Extraindo comentarios..


,token,text,csat
10,66e38bb7c92bc50e890eda22,Ótimo atendimento,5.0
15,66e37d74c92bc50e890e9f4e,RAPIDO \n,5.0
22,66e370411944e0bb7b370fcb,Agilidade no atendimento,4.0
31,66e35c66b4914b6c8e9af332,Ótimo atendimento,5.0


**Definindo funções**

In [6]:
def analyze_sentiments(text_content):

    list_docs = list()
    list_sents = list()

    # Instantiates a client
    client = language_v2.LanguageServiceClient()
    for idx in text_content.index:        
        text = text_content.loc[idx]
        if len(text) > 5:
            type_ = language_v2.Document.Type.PLAIN_TEXT
            document = {"content":text, "type_":type_}
            encoding_type = language_v2.EncodingType.UTF8
            response = client.analyze_sentiment(request = {'document':document, 'encoding_type':encoding_type})
            # document_sentiment: Sentimento do texto completo
            document_sentiment = response.document_sentiment
            magnitude = document_sentiment.magnitude
            score = document_sentiment.score
            dict_doc = {'token': idx, 'text': text, 'magnitude':magnitude, 'score':score}
            # sentences: Sentimentos por sentença do texto
            sentences = response.sentences
            for i in range(len(sentences)):        
                text = sentences[i].text.content
                magnitude = sentences[i].sentiment.magnitude
                score = sentences[i].sentiment.score
                begin_offset = 0 if i==0 else sentences[i].text.begin_offset
                dict_sent = {'token': idx, 'row':i, 'text':text, 'magnitude':magnitude, 'score':score, 'begin_offset':begin_offset}
                list_sents.append(dict_sent)
        else:
            dict_doc = {'token': idx, 'text': text, 'magnitude':0, 'score':0}
        list_docs.append(dict_doc)
    return {'document_sentiments':list_docs, 'sentences_sentiments':list_sents}

def sentiment_classify_v2(text, csat, score, magnitude):

    csat, score, magnitude = int(csat),  round(float(score), 2), round(float(magnitude), 2)    
    n_caracteres = len(text)
    # CARACTERES
    if n_caracteres < 4:
        class_ = 'Indefinido'
    elif n_caracteres <= 15:
        if csat > 3:
            class_ = 'Positivo'
        elif csat > 2:
            class_ = 'Neutro'
        else:
            class_ = 'Negativo' 
    # csat PROMOTORES
    elif csat > 3:
        # score negativo
        if score < 0:
            if score <= -0.3:
                class_ = 'Negativo'
            elif score > -0.3:
                class_ = 'Misto'
                    ## TO DO: comentário like 'fora da curva'
        # score zero
        elif score == 0:
            if magnitude == 0: 
                class_ = 'Neutro'
            else:
                class_ = 'Misto'
        # score positivo
        else:
            class_ = 'Positivo'
    # csat NEUTROS
    elif csat > 2:
        # score negativo
        if score < 0:
            class_ = 'Negativo'
        # score zero
        elif score == 0:
            if magnitude == 0:
                class_ = 'Misto'
            else:
                class_ = 'Negativo'
        # score positivo
        else:
            if score <= 0.2 and magnitude >= 0.8:
                class_ = 'Negativo'
            elif score > 0.2 and magnitude >= 0.8:
                class_ = 'Positivo'
            elif score <= 0.3:
                class_ = 'Neutro'
            elif score > 0.3 and score <= 0.5:
                class_ = 'Misto'
            else:
                class_ = 'Positivo'
    # csat DETRATORES
    elif csat <=2:
        if score >= 0.3:
            class_ = 'Misto'
        else:
            class_ = 'Negativo'
    return class_

**Classificação dos sentimentos**

In [7]:
# Supondo que df já esteja carregado e tenha as colunas ['token', 'text', 'csat']
df.columns = ['token', 'text', 'csat']

text_content = df.set_index('token').text

# Aplicando API google para analise de sentimentos 
print("Aplicando API nos comentarios..")
response = analyze_sentiments(text_content)

# Criando dataframes para sentimentos: DOC e SENT
document_sentiments = pd.DataFrame(response['document_sentiments'])
sentences_sentiments = pd.DataFrame(response['sentences_sentiments'])

# Aplicando função em DOC para extrair sentimentos
print("Identificando sentimentos..")
document_sentiments = pd.merge(document_sentiments, df[['token', 'csat']])


Aplicando API nos comentarios..


Identificando sentimentos..


In [11]:
document_sentiments['sentiment'] = document_sentiments.apply(lambda df: sentiment_classify_v2(df.text,
                                                                                              df.csat,
                                                                                              df.score,
                                                                                              df.magnitude), axis=1)

document_sentiments.head(4)


,token,text,magnitude,score,csat,sentiment
0,66e38bb7c92bc50e890eda22,Ótimo atendimento,0.947,0.870,5.0,Positivo
1,66e37d74c92bc50e890e9f4e,RAPIDO \n,0.888,0.626,5.0,Positivo
2,66e370411944e0bb7b370fcb,Agilidade no atendimento,0.929,0.853,4.0,Positivo
3,66e35c66b4914b6c8e9af332,Ótimo atendimento,0.947,0.870,5.0,Positivo
